#CS220: Assignment 4: SAT Solver Using z3
##Author: Lisa Chen
[Google Colab Link for Project](https://colab.research.google.com/drive/1ilOKsI-z8wlIL_G5gS2EJJ09rZDepyPp)

The goal is to use the z3 solver to determine if the graph coloring can be satisfied with the given graph, maximum number of colors allowed to color the graph, and minimum number of affinity edges to satisfy.

The inputs are 5 .txt files with:
* First line containing information on the graph:
  * number of vertices
  * number of interference edges
  * number of affinity edges 
  * K1 Restriction: maximum number of colors that can be used to color the graph
  * K2 Restriction: minimum number of affinity edges the solver must satisfy

Output is a text file containing: 
* "Yes" or "No" if the requirements for coloring the graph can be satisfied based on restrictions 
* If Yes:
 * One line for each vertex and what the color assignment is for each vertex


 Code was made with help from the following tutorial: 
https://ericpony.github.io/z3py-tutorial/guide-examples.htm

##**NOTE: K1 and K2**
Since the sample files did not original contain the values for K1 and K2, K1 and K2 was set based on the results of Gurobi where the chromatic number is set as K1 and K2 was the number of affinity edges Gurobi could satisfy with that given chromatic number. 

Based on this, the solver should always be satisfied; **however**, you can change these values quickly at **Part 4.1** instead of modifying the file. 

##Part 1: Set Number for File to read

In [0]:
file_num = 4

##Part 2: Setup with Google Drive and z3


###Imports and Variables

In [0]:
#z3 solver setup
pip install z3-solver

     |████████████████████████████████| 19.3MB 121kB/s 


In [0]:
import z3
from z3 import *

In [0]:
root_path = '/content/drive/My Drive/CS220/'
file_1 = root_path + 'sample_1.txt'
file_2 = root_path + 'sample_2.txt'
file_3 = root_path + 'sample_3.txt'
file_4 = root_path + 'sample_4.txt'
file_5 = root_path + 'sample_5.txt'

interference_marker = 1
affinity_marker = 2

###**IMPORTANT! Read!**
The user must place the 5 text files in your personal Google Drive in the following folder: 
/MyDrive/CS220/

Without this, the code will not work. 

In [0]:
#mount Google Drive to access files (it will have you go to a link for access)
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##Part 3: Read the file and create edge matrix representing the graph

In [0]:
switcher = {
    1: file_1,
    2: file_2,
    3: file_3,
    4: file_4,
    5: file_5
}
file_path = switcher.get(file_num, 'Invalid')

In [0]:
if (file_path != 'Invalid'):
  reader = open(file_path,'r')
  first_line = reader.readline()
  graph_params = first_line.split()
  qtyVert = int(graph_params[0])
  qtyInterference = int(graph_params[1])
  qtyAffinity = int(graph_params[2])
  maxColorsCanUse = int(graph_params[3])
  minAffinityToSatisfy = int(graph_params[4])
  print(qtyVert, qtyInterference, qtyAffinity, maxColorsCanUse, 
        minAffinityToSatisfy)
else:
  print("You put a file number that doesn't correspond to a filename.")

20 74 19 5 18


In [0]:
#create the edge matrix from the graph information
edgeMatrix = [[0 for x in range(qtyVert)] for y in range(qtyVert)]
iterator = 0
for line in reader:
  edge = line.split()
  vert1 = int(edge[0])
  vert2 = int(edge[1])
  if (iterator < qtyInterference):
    marker = interference_marker
  else:
    marker = affinity_marker
  edgeMatrix[vert1-1][vert2-1] = marker
  iterator += 1
#for debugging - print edgeMatrix
# edgeMatrix

##Part 4: Solver Variables

###Part 4.1: Modify K1 and K2 constraints here
This is a quick way to test if the solver can find a graph coloring solutions with the given max colors that the solver can use (K1) and the number of affinity edges the solver must satisfy (K2). 

In [0]:
#Current K1 and K2 for reference
print(maxColorsCanUse) #K1
print(minAffinityToSatisfy) #K2

4
18


In [0]:
#Remove comment and modify the K1 and K2 values
maxColorsCanUse = 6
minAffinityToSatisfy = 19

###Part 4.2: Set up and intialize new solver

In [0]:
s = Solver()
color_assign = [ Int('vert%s' % (i+1)) for i in range(qtyVert) ]

##Part 5: Add Constraints

In [0]:
#Constraint for max colors that can be used
max_color_constr = [ And(1 <= color_assign[vert], color_assign[vert] <= 
                         maxColorsCanUse) for vert in range(qtyVert) ]

s.add(max_color_constr)

In [0]:
aff_index = 0
satisfied_affinity = [0 for x in range(qtyAffinity)]

for vert1 in range(qtyVert):
  for vert2 in range(qtyVert):
    #Add constraint for interference edge
    if edgeMatrix[vert1][vert2] == interference_marker:
      s.add(color_assign[vert1] != color_assign[vert2])
    #counting all the satisfied affinity edges
    if edgeMatrix[vert1][vert2] == affinity_marker:
      satisfied_affinity[aff_index] = If(color_assign[vert1] == 
                                         color_assign[vert2], 1, 0)
      aff_index += 1

#add constraint for the required affinity edges to satisfy
s.add(sum(satisfied_affinity) >= minAffinityToSatisfy)

In [0]:
#for debugging through checking constraints
# s.statistics

##Part 6: Print Results
Creates an output file in the CS220 folder in your Google Drive

In [0]:
#check whether or not the solver was able to satisfy constraints
s.check()

unsat

In [0]:
output_filename = root_path + 'LC_output_' + str(file_num) + '.txt'

file_writer = open(output_filename,'w+')

if s.check() == sat:
  m = s.model()

  file_writer.write('Yes\n')
  print('Yes')
  for i in range(qtyVert):
    assignment = str( m.evaluate(color_assign[i]) )
    file_writer.write(assignment + '\n')
    print('V' + str(i+1) + ' ' + assignment)

else:
  file_writer.write('No')
  print('No')

file_writer.close()

No
